# TensorFlow ArcFace

### Ref https://github.com/peteryuX/arcface-tf2

In [1]:
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from models import ArcFaceModel
# from losses import SoftmaxLoss
from losses import softmax_loss
import dataset
import tensorflow as tf
import os
import logging

tf.get_logger().setLevel(logging.ERROR)
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"s

gpu_num = 2

os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_num)

In [2]:
!echo $CUDA_VISIBLE_DEVICES

2


In [3]:
### IJB-C Dataset
# batch_size = 128
# input_size = 112
# embd_shape = 512
# head_type = 'ArcHead'
# backbone_type = 'MobileNetV2'
# w_decay=5e-4
# num_classes = 3584 
# base_lr = 0.01
# dataset_len = 13033 
# epochs = 100
# steps_per_epoch = dataset_len // batch_size

### MS1M dataset
num_classes = 85742 
dataset_len = 5822653
batch_size = 256 # Initially 128
input_size = 112
embd_shape = 512
train_size = int(0.8 * dataset_len)
steps_per_epoch = train_size // batch_size
val_size = dataset_len - train_size
validation_steps = val_size // batch_size

w_decay=5e-4
epochs = 30

save_steps = 5000
steps = 1
is_ccrop=False
binary_img=True

is_Adam = False   # True
projection_head = False  # True
dgx = True

head_type = 'CurHead' # ''ArcHead', CosHead', 'SphereHead', 'CurHead', 'AdaHead', CadHead'  
# Backbones w/ pretrained weights:
#     MobileNet, MobileNetV2, InceptionResNetV2, InceptionV3, ResNet50, ResNet50V2, ResNet101V2, NASNetLarge, NASNetMobile, Xception, MobileNetV3Large, MobileNetV3Small, EfficientNetLite0~6, EfficientNetB0~7
# Backbones w/o pretrained weights:
#      MnasNetA1, MnasNetB1, MnasNetSmall 
backbone_type = 'EfficientNetB6' 

if head_type == 'SphereHead':
    base_lr = 0.01 
    margin = 1.35
    logist_scale = 30.0 
elif head_type == 'CosHead':
    base_lr = 0.01 
    margin=0.35
    logist_scale=64
elif head_type == 'ArcHead':
    base_lr = 0.1 
    margin=0.5
    logist_scale=64
elif head_type == 'CurHead': 
    base_lr = 0.1 
    margin=0.5
    logist_scale=64
elif head_type == 'CadHead': 
    base_lr = 0.1 
    margin=0.0
    logist_scale=64
elif head_type == 'AdaHead':
    base_lr = 0.1 
    margin=0.0
    logist_scale=64
else:
    base_lr = 0.01 # initially 0.01
    
print(head_type)
print(backbone_type)
print("projection head:", projection_head)
print("Adam:", is_Adam)
print("epoch:", epochs)
print("batch size:", batch_size)

CurHead
EfficientNetB6
projection head: False
Adam: False
epoch: 30
batch size: 256


In [4]:
# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'
if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = f'{train_data_dir}ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = f'{train_data_dir}ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, train_size=train_size, binary_img=binary_img, 
    is_ccrop=is_ccrop)

# print("data: ", train_dataset)


strategy = tf.distribute.MirroredStrategy(devices=[f"/gpu:{gpu_num}"])
# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

with strategy.scope():
    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             margin=margin, 
                             logist_scale=logist_scale,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             use_pretrain=False,
                             training=True,
                             projection_head=projection_head)
    model.summary()

    learning_rate = tf.constant(base_lr)
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(learning_rate,
                                                                 decay_steps=200000, 
                                                   decay_rate=0.1,staircase=True)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=lr_schedule)
    else:
#         optimizer = tf.keras.optimizers.SGD(
#             learning_rate=learning_rate, momentum=0.9)
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=lr_schedule, momentum=0.9)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)

Model: "arcface_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
efficientnetb6 (Functional)     (None, 4, 4, 2304)   40960136    input_image[0][0]                
__________________________________________________________________________________________________
OutputLayer (Functional)        (None, 512)          18886144    efficientnetb6[0][0]             
__________________________________________________________________________________________________
label (InputLayer)              [(None,)]            0                                            
______________________________________________________________________________________

In [5]:
from pathlib import Path
if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/"
    if projection_head:
        save_name = f'ms1m_{backbone_type}_{head_type}_ProjectionHead_check/'
    else:
        save_name = f'ms1m_{backbone_type}_{head_type}_check/'
else:
    base_dir = "/hd/honghee/models/"
    save_name = f'ms1m_{backbone_type}_{head_type}_check/'

if is_Adam:
    version = "Adam"
else:
    version = "SGD"

Path(f'{base_dir}checkpoints/w_tfidentity/{save_name}{version}').mkdir(parents=True, exist_ok=True)

### MS1M dataset
tb_callback = TensorBoard(log_dir='logs/arcface/curhead',
                                  update_freq = 1000,
                                  profile_batch=2)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
check_dir = f'{base_dir}checkpoints/w_tfidentity/{save_name}{version}'
mc_callback = ModelCheckpoint(
            check_dir+'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# callbacks = [mc_callback, tb_callback, early_stopping]
callbacks = [mc_callback, early_stopping]
# callbacks = [mc_callback, tb_callback, early_stopping, lr_scheduler]

### IJB-C Dataset
# callbacks = [
#     ModelCheckpoint(
#         base_dir+"checkpoints/"+save_name+".ckpt", 
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         verbose=1, 
#         save_best_only=True, 
#         save_weights_only = True,
#         mode='min'
#     ),
#     EarlyStopping(
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         patience=15, 
#         min_delta=0.001, 
#         mode='min'
#     )
# ]

model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=epochs, 
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 1/30
 5000/18195 [=======>......................] - ETA: 3:08:40 - loss: 56.5520
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CurHead_check/SGD/e_1_l_44.88495635986328.ckpt
10000/18195 [===============>..............] - ETA: 1:57:15 - loss: 49.5636
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CurHead_check/SGD/e_1_l_41.05544662475586.ckpt
15000/18195 [=======================>......] - ETA: 45:42 - loss: 46.4550
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CurHead_check/SGD/e_1_l_39.56924057006836.ckpt
18195/18195 [==============================] - 16300s 893ms/step - loss: 45.1854 - val_loss: 35.3256
Epoch 2/30
 1805/18195 [=>............................] - ETA: 3:58:43 - loss: 34.2017
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentit

14440/18195 [======================>.......] - ETA: 54:40 - loss: 7.8308
Epoch 00009: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CurHead_check/SGD/e_9_l_7.652403354644775.ckpt
18195/18195 [==============================] - 16495s 907ms/step - loss: 7.7804 - val_loss: 7.8347
Epoch 10/30
 1245/18195 [=>............................] - ETA: 4:05:51 - loss: 6.2905
Epoch 00010: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CurHead_check/SGD/e_10_l_6.517336368560791.ckpt
 6245/18195 [=========>....................] - ETA: 2:53:51 - loss: 6.5305
Epoch 00010: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CurHead_check/SGD/e_10_l_6.596700668334961.ckpt
11245/18195 [=================>............] - ETA: 1:41:15 - loss: 6.5393
Epoch 00010: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_Efficie

 5685/18195 [========>.....................] - ETA: 3:02:57 - loss: 3.2098
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CurHead_check/SGD/e_18_l_3.3109238147735596.ckpt
10685/18195 [================>.............] - ETA: 1:49:58 - loss: 3.2592
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CurHead_check/SGD/e_18_l_3.3107657432556152.ckpt
15685/18195 [========================>.....] - ETA: 36:44 - loss: 3.2711
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CurHead_check/SGD/e_18_l_3.278376579284668.ckpt
18195/18195 [==============================] - 16593s 912ms/step - loss: 3.2706 - val_loss: 3.7512
Epoch 19/30
 2490/18195 [===>..........................] - ETA: 3:55:31 - loss: 3.0352
Epoch 00019: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_Effi

15125/18195 [=======================>......] - ETA: 45:08 - loss: 2.6622
Epoch 00026: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CurHead_check/SGD/e_26_l_2.6690008640289307.ckpt
18195/18195 [==============================] - 16675s 916ms/step - loss: 2.6616 - val_loss: 3.2536
Epoch 27/30
 1930/18195 [==>...........................] - ETA: 3:58:30 - loss: 2.5295
Epoch 00027: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CurHead_check/SGD/e_27_l_2.5928921699523926.ckpt
 6930/18195 [==========>...................] - ETA: 2:45:20 - loss: 2.6172
Epoch 00027: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetB6_CurHead_check/SGD/e_27_l_2.6780905723571777.ckpt
11930/18195 [==================>...........] - ETA: 1:31:55 - loss: 2.6412
Epoch 00027: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_Eff

### Resume training with latest checkpoint

In [4]:
from glob import glob
from pathlib import Path

if is_Adam:
    version = "Adam"
else:
    version = "SGD"

if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/"
    # save_name = "ms1m_mobilenet_check/SGD/*"
    if projection_head:
        save_name = f'ms1m_{backbone_type}_{head_type}_ProjectionHead_check/{version}/*'
    else:
        save_name = f'ms1m_{backbone_type}_{head_type}_check/{version}/*'
else:
    base_dir = "/hd/honghee/models/checkpoints/w_tfidentity/"
    save_name = f'ms1m_{backbone_type}_{head_type}_check/{version}/*'
file_list = []
for files in glob(f'{base_dir}{save_name}'):
    if not files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] == 'checkpoint':
        loss = float( files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] )
    file_list.append( loss  )
file_list.sort()

load_file_name = []
for files in glob(f'{base_dir}{save_name}'):
    if files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] == 'checkpoint':
        pass
    elif file_list[0] == float( files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] ) and files.split('/')[-1].split('l_')[-1].split('.ckpt')[-1]!='.index':
        load_file_name = files
best_checkpoint = load_file_name.split('.data')[0]
initial_epoch = int(load_file_name.split('e_')[-1].split('_')[0])-1
print(initial_epoch)
print(best_checkpoint)

2
/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_3_l_19.344934463500977.ckpt


In [5]:
# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'

if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = f'{train_data_dir}ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = f'{train_data_dir}ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, train_size=train_size, binary_img=binary_img,
    is_ccrop=is_ccrop)

if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/"
    if projection_head:
        save_name = f'ms1m_{backbone_type}_{head_type}_ProjectionHead_check/'
    else:
        save_name = f'ms1m_{backbone_type}_{head_type}_check/'
else:
    base_dir = "/hd/honghee/models/"
    save_name = f'ms1m_{backbone_type}_{head_type}_check/'

Path(f'{base_dir}checkpoints/w_tfidentity/{save_name}{version}').mkdir(parents=True, exist_ok=True)

# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:{gpu_num}"])
strategy = tf.distribute.MirroredStrategy(devices=[f"/gpu:{gpu_num}"])

with strategy.scope():

    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True,
                             projection_head=projection_head)
    model.load_weights(best_checkpoint)

    learning_rate = tf.constant(base_lr)
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(learning_rate,
                                                                 decay_steps=300000, 
                                                   decay_rate=0.1,staircase=True)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=lr_schedule)
    else:
#         optimizer = tf.keras.optimizers.SGD(
#             learning_rate=learning_rate, momentum=0.9)
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=lr_schedule, momentum=0.9)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)
    model.summary()
    
tb_callback = TensorBoard(log_dir='logs/arcface/curhead',
                                  update_freq = 1000,
                                  profile_batch=2)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
check_dir = f'{base_dir}checkpoints/w_tfidentity/{save_name}{version}'
mc_callback = ModelCheckpoint(
            check_dir+'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

callbacks = [mc_callback, early_stopping]
# callbacks = [mc_callback, tb_callback, early_stopping]
# callbacks = [mc_callback, tb_callback]

Model: "arcface_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
resnet50 (Functional)           (None, 4, 4, 2048)   23587712    input_image[0][0]                
__________________________________________________________________________________________________
OutputLayer (Functional)        (None, 512)          16787968    resnet50[0][0]                   
__________________________________________________________________________________________________
label (InputLayer)              [(None,)]            0                                            
______________________________________________________________________________________

In [6]:
model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=epochs, 
    initial_epoch=initial_epoch,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 3/20
 1000/36391 [..............................] - ETA: 1:19:53 - loss: 17.1177
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_3_l_17.141321182250977.ckpt
 2000/36391 [>.............................] - ETA: 1:16:17 - loss: 17.1267
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_3_l_17.109325408935547.ckpt
 3000/36391 [=>............................] - ETA: 1:13:15 - loss: 17.1130
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_3_l_17.05596923828125.ckpt
 4000/36391 [==>...........................] - ETA: 1:08:11 - loss: 17.0947
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_3_l_17.030012130737305.ckpt
 5000/36391 [===>..........................] - ETA: 1:04:03 - 

36391/36391 [==============================] - 4335s 119ms/step - loss: 16.0488 - val_loss: 13.2359
Epoch 4/20
  609/36391 [..............................] - ETA: 1:05:35 - loss: 11.5173
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_4_l_11.468496322631836.ckpt
 1609/36391 [>.............................] - ETA: 1:04:13 - loss: 11.5025
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_4_l_11.478160858154297.ckpt
 2609/36391 [=>............................] - ETA: 1:02:11 - loss: 11.4803
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_4_l_11.414444923400879.ckpt
 3609/36391 [=>............................] - ETA: 59:35 - loss: 11.4551
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_che

35218/36391 [============================>.] - ETA: 2:06 - loss: 7.8365
Epoch 00005: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_5_l_7.4903645515441895.ckpt
36218/36391 [============================>.] - ETA: 18s - loss: 7.8267
Epoch 00005: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_5_l_7.470621585845947.ckpt
36391/36391 [==============================] - 4223s 116ms/step - loss: 7.8250 - val_loss: 7.2360
Epoch 6/20
  827/36391 [..............................] - ETA: 1:06:12 - loss: 6.2054
Epoch 00006: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_6_l_6.207218647003174.ckpt
 1827/36391 [>.............................] - ETA: 1:03:24 - loss: 6.2195
Epoch 00006: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_6_l_

32436/36391 [=========================>....] - ETA: 7:00 - loss: 5.2989
Epoch 00007: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_7_l_5.252306938171387.ckpt
33436/36391 [==========================>...] - ETA: 5:13 - loss: 5.2974
Epoch 00007: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_7_l_5.244982719421387.ckpt
34436/36391 [===========================>..] - ETA: 3:27 - loss: 5.2957
Epoch 00007: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_7_l_5.237680912017822.ckpt
35436/36391 [============================>.] - ETA: 1:41 - loss: 5.2940
Epoch 00007: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_7_l_5.2306084632873535.ckpt
36391/36391 [==============================] - 4158s 114ms/step - loss: 5.2923 - val_loss: 

32045/36391 [=========================>....] - ETA: 7:42 - loss: 4.6699
Epoch 00008: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_8_l_4.609582901000977.ckpt
33045/36391 [==========================>...] - ETA: 5:56 - loss: 4.6676
Epoch 00008: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_8_l_4.577266216278076.ckpt
34045/36391 [===========================>..] - ETA: 4:09 - loss: 4.6644
Epoch 00008: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_8_l_4.544215202331543.ckpt
35045/36391 [===========================>..] - ETA: 2:23 - loss: 4.6605
Epoch 00008: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_8_l_4.5117692947387695.ckpt
36045/36391 [============================>.] - ETA: 36s - loss: 4.6560
Epoch 00008: saving 

...

26481/36391 [====================>.........] - ETA: 17:59 - loss: 2.6501
Epoch 00012: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_12_l_2.701799154281616.ckpt
27481/36391 [=====================>........] - ETA: 16:09 - loss: 2.6519
Epoch 00012: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_12_l_2.6978394985198975.ckpt
28481/36391 [======================>.......] - ETA: 14:21 - loss: 2.6534
Epoch 00012: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_12_l_2.6940553188323975.ckpt
29481/36391 [=======================>......] - ETA: 12:31 - loss: 2.6548
Epoch 00012: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_12_l_2.6954338550567627.ckpt
30481/36391 [========================>.....] - ETA: 10:43 - loss: 2.6562
Epoch 00

23699/36391 [==================>...........] - ETA: 22:57 - loss: 2.5315
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_14_l_2.6026577949523926.ckpt
24699/36391 [===================>..........] - ETA: 21:07 - loss: 2.5343
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_14_l_2.6009271144866943.ckpt
25699/36391 [====================>.........] - ETA: 19:18 - loss: 2.5369
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_14_l_2.5991830825805664.ckpt
26699/36391 [=====================>........] - ETA: 17:29 - loss: 2.5392
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_14_l_2.5963375568389893.ckpt
27699/36391 [=====================>........] - ETA: 15:41 - loss: 2.5412
Epoch 0

20917/36391 [================>.............] - ETA: 27:35 - loss: 2.4218
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_16_l_2.50701642036438.ckpt
21917/36391 [=================>............] - ETA: 25:47 - loss: 2.4257
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_16_l_2.507241725921631.ckpt
22917/36391 [=================>............] - ETA: 24:00 - loss: 2.4292
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_16_l_2.507878065109253.ckpt
23917/36391 [==================>...........] - ETA: 22:15 - loss: 2.4325
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_16_l_2.5082669258117676.ckpt
24917/36391 [===================>..........] - ETA: 20:27 - loss: 2.4355
Epoch 00016

18135/36391 [=============>................] - ETA: 32:32 - loss: 2.1179
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_18_l_2.223524332046509.ckpt
19135/36391 [==============>...............] - ETA: 30:44 - loss: 2.1234
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_18_l_2.223574161529541.ckpt
20135/36391 [===============>..............] - ETA: 28:59 - loss: 2.1283
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_18_l_2.2222647666931152.ckpt
21135/36391 [================>.............] - ETA: 27:12 - loss: 2.1328
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_18_l_2.222892999649048.ckpt
22135/36391 [=================>............] - ETA: 25:26 - loss: 2.1369
Epoch 0001

15353/36391 [===========>..................] - ETA: 37:57 - loss: 2.0341
Epoch 00020: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_20_l_2.150834321975708.ckpt
16353/36391 [============>.................] - ETA: 36:10 - loss: 2.0413
Epoch 00020: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_20_l_2.15371036529541.ckpt
17353/36391 [=============>................] - ETA: 34:23 - loss: 2.0479
Epoch 00020: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_20_l_2.157562017440796.ckpt
18353/36391 [==============>...............] - ETA: 32:36 - loss: 2.0540
Epoch 00020: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_CurHead_check/SGD/e_20_l_2.1605777740478516.ckpt
19353/36391 [==============>...............] - ETA: 30:51 - loss: 2.0595
Epoch 00020